# Przetwarzanie Grafiki i Muzyki - laboratorium nr 7

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Binaryzacja.

Binaryzacja polega na zamianie obrazów monochromatycznych na obrazy binarne. Przekształcenie to jest prawie zawsze wykorzystywane w analizie obrazów, gdyż wiele operacji może być przeprowadzonych wyłącznie na obiektach binarnych.

## Zadanie 1 (1 punkt)

Proszę zaimplementować binaryzację z zadanym poziomem ( a ).

$$
J(i,j) = \left\{ \begin{array}{ll}
0 & \textrm{gdy } J(i,j)<a \\
255 & \textrm{gdy } J(i,j) \geq a 
\end{array} \right.
$$

  * Proszę zastosować do zdjęcia "LENA_512.jpg". 
  * Proszę wykonać na Luminancji (składowa Y z formatu YCbCr). 

In [23]:
def binarize_with_level(image, a):
    ycbcr_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    height, width, _ = ycbcr_image.shape
    binarized_image = np.zeros((height, width)).astype(np.uint8)
    mask = np.all(ycbcr_image[:,:,0] >= a, axis=0)
    binarized_image[mask] = 255
    return binarized_image
    

a = 100
lena = cv2.imread("./img/LENA_512.jpg", cv2.IMREAD_COLOR)
binarized_level_lena = binarize_with_level(lena, a)

cv2.imshow("binarized_level_lena", binarized_level_lena)

cv2.waitKey(50)
cv2.destroyAllWindows()

[[161 162 163 ... 169 154 127]
 [162 162 162 ... 171 157 130]
 [162 162 162 ... 169 156 128]
 ...
 [ 44  41  53 ... 102 100 100]
 [ 43  41  57 ... 104 106 106]
 [ 44  42  57 ... 102 108 109]]
(512, 512, 3)
(512, 512)
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fals

## Zadanie 2 (1 punkt) 

Proszę zaimplementować binaryzację z zadanym przedziałem ($[a,b]$):

$$
J(i,j) = \left\{ \begin{array}{ll}
255 & \textrm{gdy } J(i,j) \in \left[a,b\right] \\
0 & \textrm{w przeciwnym przypadku} 
\end{array} \right.
$$

Zastosuj tę binaryzację do obrazu "LENA_512.jpg".

In [24]:
def binarize_with_range(image, a, b):
    ycbcr_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    height, width, _ = ycbcr_image.shape
    binarized_image = np.zeros((height, width)).astype(np.uint8)
    mask = np.all((ycbcr_image[:,:,0] >= a, ycbcr_image[:,:,0] <= b), axis=0)
    binarized_image[mask] = 255
    return binarized_image

a = 100
b = 150
lena = cv2.imread("./img/LENA_512.jpg", cv2.IMREAD_COLOR)
binarized_range_lena = binarize_with_range(lena, a, b)

cv2.imshow("binarized_range_lena", binarized_range_lena)

cv2.waitKey(50)
cv2.destroyAllWindows()

# Zadanie 3 (2 punkty)

Prosze użyć algorytm binaryzacji Otsu do zdjęcia "gazeta.jpg". 

![alt](https://raw.githubusercontent.com/przem85/PGiM/master/img/gazeta.jpg)

Proszę wykonać na Luminancji (składowa Y z formatu YCbCr).

In [35]:
# binaryzacja Otsu:
# 1. obliczenie histogramu na grayscale - histogram musi być bimodalny (musi mieć dwa peaki)
#   jeden z peaków reprezentuje wartości obiektu, ktory chcemy znaleźć
#   drugi peak reprezentuje tło
#   trzeba znalezc próg, który oddzieli jeden peak od drugiego
# 2. znajdowanie progu
#   a. minimalizacja wariancji wewnątrzklasowej
#   sigma_w(T) = w_1(T) * sigma^2_1 (T) + w_2(T) * sigma^2_2(T)
#   w_1(T) = P(Y = 1) = P(i nalezy do {0,1,...,T}) = sum_j=0^T P(j)
#   w_2(T) = P(Y = 2) = P(i nalezy do {T+1,...,i_max}) = sum_j=T+1^i_max P(j)
#    - prawdopodobieństwo, że piksele należą do klasy nr 1/2 dla jakiegoś progu T
#   mi_1(T) = wartosc oczekiwana (i | Y = 1) = sum_j=0^T j * P(i = j | Y = 1) = sum_j=0^T j * P(i = j, Y = 1) / P (Y = 1) =
#    = sum 
# ...


def binarize_otsu(image):
    ycbcr_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    flag = cv2.THRESH_BINARY + cv2.THRESH_OTSU
    ret, binarized_image = cv2.threshold(ycbcr_image[:,:,0], 0, 255, flag)
    return binarized_image

lena = cv2.imread("./img/LENA_512.jpg", cv2.IMREAD_COLOR)
binarized_otsu_lena = binarize_otsu(lena)

cv2.imshow("binarized_otsu_lena", binarized_otsu_lena)

cv2.waitKey(50)
cv2.destroyAllWindows()

# Zadanie 4 (3 punkty)

Proszę zaimplementować algorytm binaryzacji Bernsena. Metoda Bernsena to algorytm wyznaczania progu lokalnego w oparciu o otoczenie piksela (zwykle kwadratowe okno o nieparzystej szerokości). 
Próg ustawiany jest na wartość średnią najjaśniejszego i najciemniejszego piksela.

  * Proszę użyć algorytm binaryzacji Bernsena do zdjęcia "gazeta.jpg".
  * Proszę wykonać na Luminancji (składowa Y z formatu YCbCr).

Algorytm Brensena:
1. Przekształcamy oryginalny obrazek do skali szarości. Możemy teraz myśleć o obrazku, jak o pewnej funkcji $I\left(x,y\right): \mathbf{N}^2 \rightarrow \mathbf{N}$ (zakładając, że poruszamy się po świecie liczb naturalnych).
2. Wokół każdego piksela bierzemy kwadrat o nieparzystej długości $a>0$ i wyliczamy wartość minimalną oraz maksymalną z intensywności pikseli w każdym takim kwadracie. Formalnie, dla każdego punktu $\left(x,y\right)\in\mathbf{N}^2$ i ustalonego $a>0$ wyliczamy:

\begin{align}
  v_{min}\left(x,y;a\right) &= \min{ \left( I\left(i,j\right): x-a \leq i \leq x+a, \quad y-a \leq j \leq y+a \right) } \\
  v_{max}\left(x,y;a\right) &= \max{ \left( I\left(i,j\right): x-a \leq i \leq x+a, \quad y-a \leq j \leq y+a \right) }
\end{align}
3. Dla każdego kwadratu wyliczamy próg $T$ jako $T = \frac{v_{min}\left(x,y;a\right) + v_{max}\left(x,y;a\right)}{2}$.

In [6]:
# 1. obrazek do grayscale
# 2. dla kazdej intensywnosci piklsela _i_ bierzemy kwadrat i wyliczamy min oraz max
# 3. 
# przyklad:
#     1 2 3 4    
#     4 3 2 1 
#     2 3 1 2    
#     4 1 1 1
# 
#     1 2 3 | 4
#     4 3 2 | 1  -> liczymy v_min i v_max z kazdego takiego kwadratu, nawet jak wychodzi poza obrazek
#     2 3 1 | 2     prog to T = (v_min + v_max) / 2
#     -------
#     4 1 1 1



def binarize_bernsen(image, window_size = 5):
    ycbcr_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    gray_channel = ycbcr_image[:,:,0]
    binarized_image = np.zeros_like(gray_channel, dtype=np.uint8)
    rows, cols = binarized_image.shape

    for i in range(rows):
        for j in range(cols):
            row_begin = max(0, i - window_size // 2)
            row_end = min(rows, i + window_size // 2 + 1)
            col_begin = max(0, j - window_size // 2)
            col_end = min(cols, j + window_size // 2 + 1)

            min_val = np.min(ycbcr_image[row_begin:row_end, col_begin:col_end])
            max_val = np.max(ycbcr_image[row_begin:row_end, col_begin:col_end])
            threshold = (min_val + max_val) // 2

            if gray_channel[i, j] > threshold:
                binarized_image[i, j] = 255
            else:
                binarized_image[i, j] = 0

    return binarized_image


gazeta = cv2.imread("./img/gazeta.jpg", cv2.IMREAD_COLOR)
bernsen_binarized_gazeta = binarize_bernsen(gazeta, 5)

cv2.imshow("original gazeta", gazeta)
cv2.imshow("bernsen binarized gazeta", bernsen_binarized_gazeta)

cv2.waitKey(50)
cv2.destroyAllWindows()

/var/folders/6t/lw9p33tn1q124mgvz_z7v9gm0000gn/T/ipykernel_4261/2949165266.py:33: RuntimeWarning: overflow encountered in scalar add
  threshold = (min_val + max_val) // 2


# Zadanie 5 (3 punkty)
Progowanie mieszane przebiega podobnie, jak progowanie lokalne (metoda Bernsena), z tym, że jeśli średnia lub mediana lokalna dla danego piksela odbiega o więcej niż ustalony próg (program pobiera tą wartość jako parametr) od wartości globalnej (wyznaczona za pomocą Otsu), piksel progowany jest wartością globalną.

  * Prosze użyć algorytm progowania mieszanego do zdjęcia "gazeta.jpg".
  * Proszę wykonać na Luminancji (składowa Y z formatu YCbCr).

  Algorytm progowania mieszanego:
  1. Wyznaczamy wartość progu globalnego przy pomocy binaryzacji Otsu. Oznaczmy ten próg jako $P$.
  2. Podobnie, jak w algorytmie Brensena, przekształcamy oryginalny obrazek do skali szarości.
  3.  Wokół każdego piksela bierzemy kwadrat o nieparzystej długości $a>0$ i wyliczamy wartość minimalną oraz maksymalną z intensywności pikseli w każdym takim kwadracie. Formalnie, dla każdego punktu $\left(x,y\right)\in\mathbf{N}^2$ i ustalonego $a>0$ wyliczamy:

\begin{align}
  v_{min}\left(x,y;a\right) &= \min{ \left( I\left(i,j\right): x-a \leq i \leq x+a, \quad y-a \leq j \leq y+a \right) } \\
  v_{max}\left(x,y;a\right) &= \max{ \left( I\left(i,j\right): x-a \leq i \leq x+a, \quad y-a \leq j \leq y+a \right) }
\end{align}

4. Dla każdego kwadratu wyliczamy próg $T$ jako $T = \frac{v_{min}\left(x,y;a\right) + v_{max}\left(x,y;a\right)}{2}$. Jeżeli jednak wartość tak wyliczonego progu $T$ różni się o więcej niż $\alpha > 0$ (parametr ten powinien być wczytywany jako argument funkcji) od progu globalnego $P$, to wtedy piksel progowany jest wartością globalną. Formalnie, jeśli $\left| T - \alpha \right| > P$, to wtedy jako próg lokalny przyjmujemy próg globalny $P$.

In [9]:
def binarize_mixed(image, alpha, window_size = 5):
    ycbcr_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    gray_channel = ycbcr_image[:,:,0]
    flag = cv2.THRESH_BINARY + cv2.THRESH_OTSU
    global_threshold, _ = cv2.threshold(gray_channel, 0, 255, flag)
    binarized_image = np.zeros_like(gray_channel, dtype=np.uint8)
    rows, cols = binarized_image.shape

    for i in range(rows):
        for j in range(cols):
            row_start = max(0, i - window_size // 2)
            row_end = min(rows, i + window_size // 2 + 1)
            col_start = max(0, j - window_size // 2)
            col_end = min(cols, j + window_size // 2 + 1)

            local_min = np.min(image[row_start:row_end, col_start:col_end])
            local_max = np.max(image[row_start:row_end, col_start:col_end])
            local_threshold = (local_min + local_max) // 2

            if abs(local_threshold - alpha) > global_threshold:
                local_threshold = global_threshold
                
            if gray_channel[i, j] > local_threshold:
                binarized_image[i, j] = 255
            else:
                binarized_image[i, j] = 0

    return binarized_image


gazeta = cv2.imread("./img/gazeta.jpg", cv2.IMREAD_COLOR)
mixed_binarized_gazeta = binarize_mixed(gazeta, 20)

cv2.imshow("original gazeta", gazeta)
cv2.imshow("mixed binarized gazeta", mixed_binarized_gazeta)

cv2.waitKey(50)
cv2.destroyAllWindows()

/var/folders/6t/lw9p33tn1q124mgvz_z7v9gm0000gn/T/ipykernel_4261/3782581073.py:18: RuntimeWarning: overflow encountered in scalar add
  local_threshold = (local_min + local_max) // 2
